# Rating data analyses

This notebook performs the following analysis steps:
* Data importing, preprocessing & cleaning
* Find out which rating scale works out for which model
    * Aggregating data by phenomenon, by model, by scale on weighted score of target option
    * Generate a plot accordingly
    * Subset the data for further analysis
* transforming raw metrics into probabilities by item by metric
    * this requires working by-phenomenon, because there are different numbers of options in the different phenomena
* combining this data into one cleaned data file with binarized options (probabilities of target vs rest)
    * long formatting of options and of metrics
* averaging probabilities across items and across seeds, by metric, by model, by phenomenon, by label type
* plotting results by metric, by phenomenon, by model, by label type

In [28]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator
import numpy as np


## Import & preprocess the data set

In [29]:
# Define the path to the 'rating' subfolder
rating_folder_path = "../results/rating"

# Recursively list all csv files in the 'free' folder
csv_files = []
for root, dirs, files in os.walk(rating_folder_path):
    for file in files:
        if file.endswith(".csv"):
            csv_files.append(os.path.join(root, file))

# Read and concatenate all CSV files
dfs = [pd.read_csv(csv_file) for csv_file in csv_files]
concatenated_df = pd.concat(dfs, ignore_index=True)

# Display the shape and a sample of the concatenated DataFrame
shape = concatenated_df.shape
sample_data = concatenated_df.head()

shape, sample_data

((10256, 17),
                  model_name  temperature  seed  item_id       phenomenon  \
 0  meta-llama/Llama-2-7b-hf          0.1     0        1  indirect_speech   
 1  meta-llama/Llama-2-7b-hf          0.1     0        1  indirect_speech   
 2  meta-llama/Llama-2-7b-hf          0.1     0        1  indirect_speech   
 3  meta-llama/Llama-2-7b-hf          0.1     0        1  indirect_speech   
 4  meta-llama/Llama-2-7b-hf          0.1     0        2  indirect_speech   
 
                                               prompt  \
 0  You will read a short story that describes an ...   
 1  You will read a short story that describes an ...   
 2  You will read a short story that describes an ...   
 3  You will read a short story that describes an ...   
 4  You will read a short story that describes an ...   
 
                                         prior_prompt question  \
 0  You will read a short story that describes an ...      NaN   
 1  You will read a short story that describes

In [30]:
# Checking for NA values in the concatenated DataFrame
na_values = concatenated_df.isnull().sum()

na_values


model_name                       0
temperature                      0
seed                             0
item_id                          0
phenomenon                       0
prompt                           0
prior_prompt                     0
question                      7760
options                          0
option_names                     0
scale                            0
rating_options                   0
weighted_options                 0
chosen_option                    0
token_cond_log_probs             0
prior_token_log_probs            0
null_prior_token_log_probs       0
dtype: int64

In [31]:
# Grouping by the specified columns and calculating mean and standard deviation for 'weighted_options'
grouped_df = concatenated_df.groupby(['phenomenon', 'option_names', 'item_id', 'model_name', 'scale'])['weighted_options'].agg(['mean'])

grouped_df.reset_index(inplace=True)
grouped_df.head()


print(grouped_df.shape)
print(grouped_df.isnull().sum())
print(grouped_df.head())

(10256, 6)
phenomenon      0
option_names    0
item_id         0
model_name      0
scale           0
mean            0
dtype: int64
  phenomenon option_names  item_id              model_name        scale  \
0  coherence   competitor        1       google/flan-t5-xl  appropriate   
1  coherence   competitor        1       google/flan-t5-xl       likely   
2  coherence   competitor        1       google/flan-t5-xl    plausible   
3  coherence   competitor        1       google/flan-t5-xl     possible   
4  coherence   competitor        1  gpt-3.5-turbo-instruct  appropriate   

       mean  
0  3.147568  
1  2.909551  
2  3.006738  
3  2.923203  
4  2.769090  


## Plot of weighted score of target response against rating scales grouped by phenomenon and model

In [32]:
# Filtering the data for rows where option_names is 'target'
target_data = concatenated_df[concatenated_df['option_names'] == 'target']

# Grouping by scale, model_name, and phenomenon and calculating mean and standard deviation for weighted_options
grouped_target_data = target_data.groupby(['scale', 'model_name', 'phenomenon'])['weighted_options'].agg(['mean', 'std']).reset_index()

print(grouped_target_data.head())


         scale         model_name       phenomenon      mean       std
0  appropriate  google/flan-t5-xl        coherence  2.998544  0.086520
1  appropriate  google/flan-t5-xl          deceits  2.898867  0.110442
2  appropriate  google/flan-t5-xl           humour  2.789100  0.134618
3  appropriate  google/flan-t5-xl  indirect_speech  2.989182  0.140188
4  appropriate  google/flan-t5-xl            irony  2.985433  0.114254


In [33]:
# Extracting unique models and phenomena based on the grouped data
unique_models = grouped_df['model_name'].unique()
unique_phenomena = grouped_df['phenomenon'].unique()

# Use a custom color palette to highlight 'target' response_option
palette = {response: 'gray' if response != 'target' else 'red' for response in grouped_df['option_names'].unique()}

# Manually creating a grid of subplots
fig, axes = plt.subplots(len(unique_phenomena), len(unique_models), figsize=(20, 40), sharey=True)

# Iterating over each model and phenomenon to create individual subplots
for i, phenomenon in enumerate(unique_phenomena):
    for j, model in enumerate(unique_models):
        ax = axes[i, j]
        subset = grouped_target_data[(grouped_target_data['model_name'] == model) & 
                                     (grouped_target_data['phenomenon'] == phenomenon)]
        
        # Determining the bar color based on the highest mean
        colors = ['red' if val == subset['mean'].max() else 'gray' for val in subset['mean']]
        
        ax.bar(subset['scale'], subset['mean'], color=colors, yerr=subset['std'])
        ax.set_title(f'Model: {model} | Phenomenon: {phenomenon}')
        ax.set_xlabel('')
        ax.set_ylabel('Mean Weighted Options')

        # Use FixedLocator to set ticks
        ax.xaxis.set_major_locator(FixedLocator(range(len(subset['scale']))))
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

       # Remove all lines on background
        ax.grid(False)
        #sns.despine(left=True, bottom=True, ax=ax)

        # Set title with larger font size
        ax.set_title(f"{model} - {phenomenon}", fontsize=16)
        
        # Increase tick label size
        ax.tick_params(axis="x", rotation=90, labelsize=14)
        ax.tick_params(axis="y", labelsize=14)
        
        # Increase axis label size
        ax.set_xlabel(ax.get_xlabel(), fontsize=14)
        ax.set_ylabel(ax.get_ylabel(), fontsize=14)

        
# Adjust layout
fig.tight_layout()
plt.show()

In [34]:
# Define the path to save the plot as PDF
save_path = "plots/rating_w.score_against_scale_by_model_phenomenon.pdf"

# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the plot to the specified path
fig.savefig(save_path, bbox_inches='tight')

save_path


'plots/rating_w.score_against_scale_by_model_phenomenon.pdf'

In [35]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Fit the model for all three factors
model = ols('weighted_options ~ scale * phenomenon * model_name', data=target_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# There is no sig. three way interaction, subset the data for two way interaction

# Subset the data in models
target_data_t5 = target_data[target_data['model_name'] == 'google/flan-t5-xl']
target_data_gpt = target_data[target_data['model_name'] == 'gpt-3.5-turbo-instruct']
target_data_llama = target_data[target_data['model_name'] == 'meta-llama/Llama-2-7b-hf']

# Fit the model respectively
model_t5 = ols('weighted_options ~ scale * phenomenon', data=target_data_t5).fit()
model_gpt = ols('weighted_options ~ scale * phenomenon', data=target_data_gpt).fit()
model_llama = ols('weighted_options ~ scale * phenomenon', data=target_data_llama).fit()

# Perform ANOVA and print the table
anova_table_t5 = sm.stats.anova_lm(model_t5, typ=2)
print(anova_table_t5)

anova_table_gpt = sm.stats.anova_lm(model_gpt, typ=2)
print(anova_table_gpt)

anova_table_llama = sm.stats.anova_lm(model_llama, typ=2)
print(anova_table_llama)


                                  sum_sq      df            F         PR(>F)
scale                           7.345369     3.0     7.711263   3.969885e-05
phenomenon                    139.834090     6.0    73.399811   8.494238e-85
model_name                   1181.347644     3.0  1240.193912   0.000000e+00
scale:phenomenon                5.744996    18.0     1.005195   4.499853e-01
scale:model_name               30.543161     9.0    10.688201   1.954874e-16
phenomenon:model_name         184.023056    18.0    32.198294  6.818646e-100
scale:phenomenon:model_name    11.114997    54.0     0.648259   9.782972e-01
Residual                      823.003850  2592.0          NaN            NaN
                     sum_sq     df          F        PR(>F)
scale              1.613715    3.0  28.208081  3.743955e-17
phenomenon         2.210899    6.0  19.323491  8.715447e-21
scale:phenomenon   1.918512   18.0   5.589332  2.082910e-12
Residual          12.356832  648.0        NaN           NaN
       

## Plot of argmax weighted score against option names grouped by phenomenon and model

In [36]:
# Define conditions to subset the df for further analysis
conditions = [
    (concatenated_df['model_name'] == "google/flan-t5-xl") & (concatenated_df['scale'] == "likely"),
    (concatenated_df['model_name'] == "gpt-3.5-turbo-instruct") & (concatenated_df['scale'] == "possible"),
    (concatenated_df['model_name'] == "meta-llama/Llama-2-7b-hf") & (concatenated_df['scale'] == "appropriate"),
    (concatenated_df['model_name'] == "text-davinci-002") & (concatenated_df['scale'] == "possible")
]

combined_conditions = conditions[0] | conditions[1] | conditions[2] | conditions[3]

# Apply the subsetting conditions based on model_name and scale
df_no_scale = concatenated_df[combined_conditions]

# Grouping by model_name, and phenomenon and calculating mean and standard deviation for weighted_options
grouped_df_no_scale = df_no_scale.groupby(['model_name', 'phenomenon', 'option_names'])['weighted_options'].agg(['mean', 'std']).reset_index()



In [37]:
# Manually creating a grid of subplots
fig, axes = plt.subplots(len(unique_phenomena), len(unique_models), figsize=(20, 40), sharey=True)

# Iterating over each model and phenomenon to create individual subplots
for i, phenomenon in enumerate(unique_phenomena):
    for j, model in enumerate(unique_models):
        ax = axes[i, j]
        subset = grouped_df_no_scale[(grouped_df_no_scale['model_name'] == model) & 
                                     (grouped_df_no_scale['phenomenon'] == phenomenon)]
        
        # Determining the bar color based on the highest mean
        colors = ['red' if val == subset['mean'].max() else 'gray' for val in subset['mean']]
        
        ax.bar(subset['option_names'], subset['mean'], color=colors, yerr=subset['std'])
        ax.set_title(f'Model: {model} | Phenomenon: {phenomenon}')
        ax.set_xlabel('')
        ax.set_ylabel('Mean Weighted Options')

        # Use FixedLocator to set ticks
        ax.xaxis.set_major_locator(FixedLocator(range(len(subset['option_names']))))
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

       # Remove all lines on background
        ax.grid(False)
        #sns.despine(left=True, bottom=True, ax=ax)

        # Set title with larger font size
        ax.set_title(f"{model} - {phenomenon}", fontsize=16)
        
        # Increase tick label size
        ax.tick_params(axis="x", rotation=90, labelsize=14)
        ax.tick_params(axis="y", labelsize=14)
        
        # Increase axis label size
        ax.set_xlabel(ax.get_xlabel(), fontsize=14)
        ax.set_ylabel(ax.get_ylabel(), fontsize=14)

        
# Adjust layout
fig.tight_layout()
plt.show()

In [38]:
# Define the path to save the plot as PDF
save_path = "plots/rating_argmax.w.score_against_options_by_model_phenomenon.pdf"

# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the plot to the specified path
fig.savefig(save_path, bbox_inches='tight')

save_path


'plots/rating_argmax.w.score_against_options_by_model_phenomenon.pdf'

## Plot of weigted score of target option against option categories grouped by phenomenon and model

In [39]:
# Set up the grid layout for the subplots
fig, axes = plt.subplots(len(unique_phenomena), len(unique_models), figsize=(20, 40), sharey=True)

# Iterate over each model and phenomenon to create individual subplots
for i, phenomenon in enumerate(unique_phenomena):
    for j, model in enumerate(unique_models):
        ax = axes[i, j]
        subset = grouped_df_no_scale[(grouped_df_no_scale['model_name'] == model) & (grouped_df_no_scale['phenomenon'] == phenomenon)]
        if not subset.empty:
            sns.barplot(x='option_names', y='mean', data=subset, ax=ax, palette=palette, hue='option_names')
            ax.set_title(f'Model: {model} | Phenomenon: {phenomenon}')
            ax.set_xlabel('')
            ax.set_ylabel('')

            # Use FixedLocator to set ticks
            ax.xaxis.set_major_locator(FixedLocator(range(len(subset['option_names']))))
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
            ax.grid(True, axis='y')

            # Remove all lines on background
            ax.grid(False)
            #sns.despine(left=True, bottom=True, ax=ax)

            # Set title with larger font size
            ax.set_title(f"{model} - {phenomenon}", fontsize=16)
            
            # Increase tick label size
            ax.tick_params(axis="x", rotation=90, labelsize=14)
            ax.tick_params(axis="y", labelsize=14)
            
            # Increase axis label size
            ax.set_xlabel(ax.get_xlabel(), fontsize=14)
            ax.set_ylabel(ax.get_ylabel(), fontsize=14)

# Adjust layout
fig.tight_layout()
plt.show()


In [40]:
# Define the path to save the plot as PDF
save_path = "plots/rating_target_w.score_against_options_by_model_phenomenon.pdf"

# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the plot to the specified path
fig.savefig(save_path, bbox_inches='tight')

save_path


'plots/rating_target_w.score_against_options_by_model_phenomenon.pdf'

## Plot of proportion of choosen target against option categories grouped by phenomenon and model

In [41]:
# Use a custom color palette to highlight 'target' response_option
palette = {response: 'gray' if response != 'target' else 'red' for response in df_no_scale['chosen_option'].unique()}

# Calculating the proportion of each chosen_option for each model and phenomenon combination
proportion_data = df_no_scale.groupby(['model_name', 'phenomenon', 'chosen_option']).size().reset_index(name='counts')

# Calculating the proportion of each chosen_option for each model and phenomenon combination again
total_counts = proportion_data.groupby(['model_name', 'phenomenon'])['counts'].transform('sum')
proportion_data['proportion'] = proportion_data['counts'] / total_counts

# Plotting the proportions
fig, axes = plt.subplots(len(unique_phenomena), len(unique_models), figsize=(20, 40), sharey=True)

for i, phenomenon in enumerate(unique_phenomena):
    for j, model in enumerate(unique_models):
        ax = axes[i, j]
        subset = proportion_data[(proportion_data['model_name'] == model) & (proportion_data['phenomenon'] == phenomenon)]
        if not subset.empty:
            sns.barplot(x='chosen_option', y='proportion', data=subset, ax=ax, hue='chosen_option', palette=palette)
            ax.set_title(f'Model: {model} | Phenomenon: {phenomenon}')
            ax.set_xlabel('')
            ax.set_ylabel('')

            
            # Use FixedLocator to set ticks
            ax.xaxis.set_major_locator(FixedLocator(range(len(subset['chosen_option']))))
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
            
            # Remove all lines on background
            ax.grid(False)
            #sns.despine(left=True, bottom=True, ax=ax)

            # Set title with larger font size
            ax.set_title(f"{model} - {phenomenon}", fontsize=16)
            
            # Increase tick label size
            ax.tick_params(axis="x", rotation=90, labelsize=14)
            ax.tick_params(axis="y", labelsize=14)
            
            # Increase axis label size
            ax.set_xlabel(ax.get_xlabel(), fontsize=14)
            ax.set_ylabel(ax.get_ylabel(), fontsize=14)

# Adjust layout
fig.tight_layout()
plt.show()


In [42]:
# Define the path to save the plot as PDF
save_path = "plots/rating_propo.target_against_options_by_model_phenomenon.pdf"

# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the plot to the specified path
fig.savefig(save_path, bbox_inches='tight')

save_path


'plots/rating_propo.target_against_options_by_model_phenomenon.pdf'

## Plot of proportion of choosen target against target vs. others grouped by phenomenon and model

In [62]:
# Re-encode the option_names column to be more readable
df_no_scale_binary = df_no_scale.copy()
df_no_scale_binary.loc[:,'chosen_option'] = np.where(df_no_scale['chosen_option'] == 'target', 1, 0)

# Use a custom color palette to highlight 'target' response_option
palette = {response: 'gray' if response != 'target' else 'red' for response in df_no_scale_binary['chosen_option'].unique()}

# Calculating the proportion of each chosen_option for each model and phenomenon combination
proportion_data = df_no_scale_binary.groupby(['model_name', 'phenomenon', 'chosen_option']).size().reset_index(name='counts')

# Calculating the proportion of each chosen_option for each model and phenomenon combination again
total_counts = proportion_data.groupby(['model_name', 'phenomenon'])['counts'].transform('sum')
proportion_data['proportion'] = proportion_data['counts'] / total_counts

In [44]:

# Plotting the proportions
fig, axes = plt.subplots(len(unique_phenomena), len(unique_models), figsize=(20, 40), sharey=True)

for i, phenomenon in enumerate(unique_phenomena):
    for j, model in enumerate(unique_models):
        ax = axes[i, j]
        subset = proportion_data[(proportion_data['model_name'] == model) & (proportion_data['phenomenon'] == phenomenon)]
        if not subset.empty:
            sns.barplot(x='chosen_option', y='proportion', data=subset, ax=ax, hue='chosen_option', palette=palette)
            ax.set_title(f'Model: {model} | Phenomenon: {phenomenon}')
            ax.set_xlabel('')
            ax.set_ylabel('')

            
            # Use FixedLocator to set ticks
            ax.xaxis.set_major_locator(FixedLocator(range(len(subset['chosen_option']))))
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
            
            # Remove all lines on background
            ax.grid(False)
            #sns.despine(left=True, bottom=True, ax=ax)

            # Set title with larger font size
            ax.set_title(f"{model} - {phenomenon}", fontsize=16)
            
            # Increase tick label size
            ax.tick_params(axis="x", rotation=90, labelsize=14)
            ax.tick_params(axis="y", labelsize=14)
            
            # Increase axis label size
            ax.set_xlabel(ax.get_xlabel(), fontsize=14)
            ax.set_ylabel(ax.get_ylabel(), fontsize=14)

# Adjust layout
fig.tight_layout()
plt.show()


In [45]:
# Define the path to save the plot as PDF
save_path = "plots/rating_propo.target_against_others_by_model_phenomenon.pdf"

# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the plot to the specified path
fig.savefig(save_path, bbox_inches='tight')

save_path


'plots/rating_propo.target_against_others_by_model_phenomenon.pdf'

## Export CSV for further analysis

In [46]:
# Take a look
df_no_scale_binary.head()

model_name  temperature  seed  item_id phenomenon  \
80  meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
81  meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
82  meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
83  meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
84  meta-llama/Llama-2-7b-hf          0.1     0        2    deceits   

                                               prompt  \
80  You will read a short story that describes an ...   
81  You will read a short story that describes an ...   
82  You will read a short story that describes an ...   
83  You will read a short story that describes an ...   
84  You will read a short story that describes an ...   

                                         prior_prompt question  \
80  You will read a short story that describes an ...      NaN   
81  You will read a short story that describes an ...      NaN   
82  You will read a short story that describes an ...      NaN   
83  You will read a short story that describes an ...      NaN   
84  You will read a short story that describes an ...      NaN   

                                              options  \
80  He does not want to get into trouble for not s...   
81                 He has been reading for some time.   
82  He wants his mom to believe that he has been w...   
83  He does not want to offend his mom by not read...   
84  He does not want to hurt Monica by saying that...   

                   option_names        scale  \
80                       target  appropriate   
81            incorrect_literal  appropriate   
82    incorrect_lexical_overlap  appropriate   
83  incorrect_social_convention  appropriate   
84                       target  appropriate   

                                       rating_options  weighted_options  \
80  very inappropriate, inappropriate, neutral, ap...          1.883455   
81  very inappropriate, inappropriate, neutral, ap...          1.931843   
82  very inappropriate, inappropriate, neutral, ap...          1.924816   
83  very inappropriate, inappropriate, neutral, ap...          1.940945   
84  very inappropriate, inappropriate, neutral, ap...          1.947396   

   chosen_option                               token_cond_log_probs  \
80        others  [[-10.325904846191406, -4.785717964172363], [-...   
81        others  [[-10.107025146484375, -4.639193058013916], [-...   
82        others  [[-10.181174278259277, -4.870183944702148], [-...   
83        others  [[-10.358108520507812, -4.807172775268555], [-...   
84        others  [[-9.394673347473145, -4.746034622192383], [-9...   

                                prior_token_log_probs  \
80  [[-13.373932838439941, -2.9205374717712402], [...   
81  [[-13.373932838439941, -2.9205374717712402], [...   
82  [[-13.373932838439941, -2.9205374717712402], [...   
83  [[-13.373932838439941, -2.9205374717712402], [...   
84  [[-13.373932838439941, -2.9205374717712402], [...   

                           null_prior_token_log_probs  
80  [[-10.150535583496094, -5.103446960449219], [-...  
81  [[-10.634095191955566, -5.153406620025635], [-...  
82  [[-10.855224609375, -4.926755905151367], [-10....  
83  [[-10.363120079040527, -4.919028282165527], [-...  
84  [[-10.561054229736328, -4.877218723297119], [-...

In [72]:
# Add a column to indicate metrics
df_no_scale_binary['metric'] = 'rating'
# Define conditions to subset the df for further analysis
conditions = [
    (df_no_scale_binary['model_name'] == "google/flan-t5-xl") & (df_no_scale_binary['scale'] == "likely"),
    (df_no_scale_binary['model_name'] == "gpt-3.5-turbo-instruct") & (df_no_scale_binary['scale'] == "possible"),
    (df_no_scale_binary['model_name'] == "meta-llama/Llama-2-7b-hf") & (df_no_scale_binary['scale'] == "appropriate"),
    (df_no_scale_binary['model_name'] == "text-davinci-002") & (df_no_scale_binary['scale'] == "possible")
]

combined_conditions = conditions[0] | conditions[1] | conditions[2] | conditions[3]

# Apply the subsetting conditions based on model_name and scale
df_no_scale_binary = df_no_scale_binary[combined_conditions]


In [74]:
df_no_scale_binary

model_name  temperature  seed  item_id phenomenon  \
80    meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
81    meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
82    meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
83    meta-llama/Llama-2-7b-hf          0.1     0        1    deceits   
84    meta-llama/Llama-2-7b-hf          0.1     0        2    deceits   
...                        ...          ...   ...      ...        ...   
9719         google/flan-t5-xl          0.1     0       20   metaphor   
9720         google/flan-t5-xl          0.1     0       20   metaphor   
9721         google/flan-t5-xl          0.1     0       20   metaphor   
9722         google/flan-t5-xl          0.1     0       20   metaphor   
9723         google/flan-t5-xl          0.1     0       20   metaphor   

                                                 prompt  \
80    You will read a short story that describes an ...   
81    You will read a short story that describes an ...   
82    You will read a short story that describes an ...   
83    You will read a short story that describes an ...   
84    You will read a short story that describes an ...   
...                                                 ...   
9719  You will read a short story that describes an ...   
9720  You will read a short story that describes an ...   
9721  You will read a short story that describes an ...   
9722  You will read a short story that describes an ...   
9723  You will read a short story that describes an ...   

                                           prior_prompt question  \
80    You will read a short story that describes an ...      NaN   
81    You will read a short story that describes an ...      NaN   
82    You will read a short story that describes an ...      NaN   
83    You will read a short story that describes an ...      NaN   
84    You will read a short story that describes an ...      NaN   
...                                                 ...      ...   
9719  You will read a short story that describes an ...      NaN   
9720  You will read a short story that describes an ...      NaN   
9721  You will read a short story that describes an ...      NaN   
9722  You will read a short story that describes an ...      NaN   
9723  You will read a short story that describes an ...      NaN   

                                                options  \
80    He does not want to get into trouble for not s...   
81                   He has been reading for some time.   
82    He wants his mom to believe that he has been w...   
83    He does not want to offend his mom by not read...   
84    He does not want to hurt Monica by saying that...   
...                                                 ...   
9719                     They are meant to be together.   
9720               He would love some pasta for dinner.   
9721                        They enjoy Italian cuisine.   
9722  The tomato sauce that she makes goes well with...   
9723                    His fiance has bright red lips.   

                     option_names        scale  \
80                         target  appropriate   
81              incorrect_literal  appropriate   
82      incorrect_lexical_overlap  appropriate   
83    incorrect_social_convention  appropriate   
84                         target  appropriate   
...                           ...          ...   
9719                       target       likely   
9720                   competitor       likely   
9721  distractor_plausibleliteral       likely   
9722           distractor_literal       likely   
9723       distractor_nonsequitut       likely   

                                         rating_options  weighted_options  \
80    very inappropriate, inappropriate, neutral, ap...          1.883455   
81    very inappropriate, inappropriate, neutral, ap...          1.931843   
82    very inappropriate, inappropriate, neutral, ap...          1.924816   

In [66]:
rating_out_wide = df_no_scale_binary[['chosen_option', 'phenomenon', 'model_name', 'item_id', 'seed', 'metric']].copy()
print(rating_out_wide)

     chosen_option phenomenon                model_name  item_id  seed  metric
80               0    deceits  meta-llama/Llama-2-7b-hf        1     0  rating
81               0    deceits  meta-llama/Llama-2-7b-hf        1     0  rating
82               0    deceits  meta-llama/Llama-2-7b-hf        1     0  rating
83               0    deceits  meta-llama/Llama-2-7b-hf        1     0  rating
84               0    deceits  meta-llama/Llama-2-7b-hf        2     0  rating
...            ...        ...                       ...      ...   ...     ...
9719             0   metaphor         google/flan-t5-xl       20     0  rating
9720             0   metaphor         google/flan-t5-xl       20     0  rating
9721             0   metaphor         google/flan-t5-xl       20     0  rating
9722             0   metaphor         google/flan-t5-xl       20     0  rating
9723             0   metaphor         google/flan-t5-xl       20     0  rating

[2564 rows x 6 columns]


In [67]:
# Save csv file
wide_out_path = "processed_data/rating_accuracy_wide.csv"

# Ensure the directory exists
os.makedirs(os.path.dirname(wide_out_path), exist_ok=True)

# Save the csv to the specified path
rating_out_wide.to_csv(wide_out_path, index=False)

In [68]:
proportion_data

model_name       phenomenon  chosen_option  counts  \
0          google/flan-t5-xl        coherence              0      42   
1          google/flan-t5-xl        coherence              1      38   
2          google/flan-t5-xl          deceits              0      76   
3          google/flan-t5-xl          deceits              1       4   
4          google/flan-t5-xl           humour              0      95   
5          google/flan-t5-xl           humour              1      30   
6          google/flan-t5-xl  indirect_speech              0      80   
7          google/flan-t5-xl            irony              0      52   
8          google/flan-t5-xl            irony              1      48   
9          google/flan-t5-xl           maxims              0      68   
10         google/flan-t5-xl           maxims              1       8   
11         google/flan-t5-xl         metaphor              0      75   
12         google/flan-t5-xl         metaphor              1      25   
13    gpt-3.5-turbo-instruct        coherence              0      50   
14    gpt-3.5-turbo-instruct        coherence              1      30   
15    gpt-3.5-turbo-instruct          deceits              0      32   
16    gpt-3.5-turbo-instruct          deceits              1      48   
17    gpt-3.5-turbo-instruct           humour              0      75   
18    gpt-3.5-turbo-instruct           humour              1      50   
19    gpt-3.5-turbo-instruct  indirect_speech              0      24   
20    gpt-3.5-turbo-instruct  indirect_speech              1      56   
21    gpt-3.5-turbo-instruct            irony              0      56   
22    gpt-3.5-turbo-instruct            irony              1      44   
23    gpt-3.5-turbo-instruct           maxims              0      44   
24    gpt-3.5-turbo-instruct           maxims              1      32   
25    gpt-3.5-turbo-instruct         metaphor              0      30   
26    gpt-3.5-turbo-instruct         metaphor              1      70   
27  meta-llama/Llama-2-7b-hf        coherence              0      40   
28  meta-llama/Llama-2-7b-hf        coherence              1      40   
29  meta-llama/Llama-2-7b-hf          deceits              0      80   
30  meta-llama/Llama-2-7b-hf           humour              0      95   
31  meta-llama/Llama-2-7b-hf           humour              1      30   
32  meta-llama/Llama-2-7b-hf  indirect_speech              0      68   
33  meta-llama/Llama-2-7b-hf  indirect_speech              1      12   
34  meta-llama/Llama-2-7b-hf            irony              0      64   
35  meta-llama/Llama-2-7b-hf            irony              1      36   
36  meta-llama/Llama-2-7b-hf           maxims              0      60   
37  meta-llama/Llama-2-7b-hf           maxims              1      16   
38  meta-llama/Llama-2-7b-hf         metaphor              0      85   
39  meta-llama/Llama-2-7b-hf         metaphor              1      15   
40          text-davinci-002        coherence              0      40   
41          text-davinci-002        coherence              1      40   
42          text-davinci-002          deceits              0      44   
43          text-davinci-002          deceits              1      36   
44          text-davinci-002           humour              0      55   
45          text-davinci-002           humour              1      70   
46          text-davinci-002  indirect_speech              0      12   
47          text-davinci-002  indirect_speech              1      68   
48          text-davinci-002            irony              0      80   
49          text-davinci-002            irony              1      20   
50          text-davinci-002           maxims              0      52   
51          text-davinci-002           maxims              1      24   
52          text-davinci-002         metaphor              0      40   
53          text-davinci-002         metaphor              1      60   

    proportion  
0     0.525000  
1     0.475000  
2     

In [69]:
# Rename the columns of model_data
model_data_cleaned = proportion_data.copy()

# Subset target data
model_data_cleaned = model_data_cleaned[model_data_cleaned['chosen_option'] == 1]

model_data_cleaned.rename(columns={
    "proportion": "dependent_variable"
}, inplace=True)

# Add one new column with specified values
model_data_cleaned.loc[:,'metric'] = 'rating'

# Subset data by temperature
model_data_cleaned.drop(columns=['counts', 'chosen_option'], inplace=True)


model_data_cleaned.reset_index(drop=True, inplace=True)
model_data_cleaned

model_name       phenomenon  dependent_variable  metric
0          google/flan-t5-xl        coherence            0.475000  rating
1          google/flan-t5-xl          deceits            0.050000  rating
2          google/flan-t5-xl           humour            0.240000  rating
3          google/flan-t5-xl            irony            0.480000  rating
4          google/flan-t5-xl           maxims            0.105263  rating
5          google/flan-t5-xl         metaphor            0.250000  rating
6     gpt-3.5-turbo-instruct        coherence            0.375000  rating
7     gpt-3.5-turbo-instruct          deceits            0.600000  rating
8     gpt-3.5-turbo-instruct           humour            0.400000  rating
9     gpt-3.5-turbo-instruct  indirect_speech            0.700000  rating
10    gpt-3.5-turbo-instruct            irony            0.440000  rating
11    gpt-3.5-turbo-instruct           maxims            0.421053  rating
12    gpt-3.5-turbo-instruct         metaphor            0.700000  rating
13  meta-llama/Llama-2-7b-hf        coherence            0.500000  rating
14  meta-llama/Llama-2-7b-hf           humour            0.240000  rating
15  meta-llama/Llama-2-7b-hf  indirect_speech            0.150000  rating
16  meta-llama/Llama-2-7b-hf            irony            0.360000  rating
17  meta-llama/Llama-2-7b-hf           maxims            0.210526  rating
18  meta-llama/Llama-2-7b-hf         metaphor            0.150000  rating
19          text-davinci-002        coherence            0.500000  rating
20          text-davinci-002          deceits            0.450000  rating
21          text-davinci-002           humour            0.560000  rating
22          text-davinci-002  indirect_speech            0.850000  rating
23          text-davinci-002            irony            0.200000  rating
24          text-davinci-002           maxims            0.315789  rating
25          text-davinci-002         metaphor            0.600000  rating

In [70]:
# Save csv file
csv_out_path = "processed_data/rating_accuracy_summary.csv"
wide_out_path = "processed_data/rating_accuracy_wide.csv"


# Ensure the directory exists
os.makedirs(os.path.dirname(csv_out_path), exist_ok=True)

# Save the csv to the specified path
model_data_cleaned.to_csv(csv_out_path, index=False)

In [49]:
# coherence_not_target_df = concatenated_df[(concatenated_df['phenomenon'] == 'coherence') & 
#                                           (concatenated_df['chosen_option'] != 'target')]

# coherence_not_target_df

In [50]:
# # Plotting the proportions
# fig, axes = plt.subplots(len(unique_phenomena), len(unique_models), figsize=(20, 40), sharey=True)

# for i, phenomenon in enumerate(unique_phenomena):
#     for j, model in enumerate(unique_models):
#         ax = axes[i, j]
#         subset = proportion_data[(proportion_data['model_name'] == model) & (proportion_data['phenomenon'] == phenomenon)]
#         if not subset.empty:
#             sns.barplot(x='chosen_option', y='proportion', data=subset, ax=ax, palette=palette, hue='chosen_option')
#             ax.set_title(f'Model: {model} | Phenomenon: {phenomenon}')
#             ax.set_xlabel('')
#             ax.set_ylabel('')

            
#             # Use FixedLocator to set ticks
#             ax.xaxis.set_major_locator(FixedLocator(range(len(subset['chosen_option']))))
#             ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
            
#             # Remove all lines on background
#             ax.grid(False)
#             #sns.despine(left=True, bottom=True, ax=ax)

#             # Set title with larger font size
#             ax.set_title(f"{model} - {phenomenon}", fontsize=16)
            
#             # Increase tick label size
#             ax.tick_params(axis="x", rotation=90, labelsize=14)
#             ax.tick_params(axis="y", labelsize=14)
            
#             # Increase axis label size
#             ax.set_xlabel(ax.get_xlabel(), fontsize=14)
#             ax.set_ylabel(ax.get_ylabel(), fontsize=14)

# # Adjust layout
# fig.tight_layout()
# plt.show()

In [51]:
# # Creating a proper name for the plot
# plot_name = "rating_w.score_by_scale_model_phenomenon.pdf"
# plot_path = os.path.join("plots", plot_name)

# # Ensure the plots directory exists
# os.makedirs(os.path.dirname(plot_path), exist_ok=True)

# # Save the plot
# fig.savefig(plot_path)

# plot_path


In [52]:
# # Subsetting the DataFrame based on the specified criteria
# subset_df = concatenated_df[(concatenated_df['model_name'] == 'gpt-3.5-turbo-instruct') & 
#                             (concatenated_df['scale'] == 'possible')]

# # Grouping the subsetted data by the specified columns and calculating the mean for 'weighted_options'
# grouped_subset_df = subset_df.groupby(['phenomenon', 'option_names', 'item_id', 'model_name', 'scale'])['weighted_options'].agg(['mean']).reset_index()

# grouped_subset_df.head()

# print(grouped_subset_df.shape)
# print(grouped_subset_df.isnull().sum())
# print(grouped_subset_df.head())

In [53]:
# # Extracting unique phenomena from the grouped_subset_df DataFrame
# unique_phenomena_subset = grouped_subset_df['phenomenon'].unique()

# # Use a custom color palette for different option_names
# #palette_subset = sns.color_palette("viridis", len(grouped_subset_df['option_names'].unique()))


# # Use a custom color palette to highlight 'target' response_option
# palette = {response: 'gray' if response != 'target' else 'red' for response in grouped_subset_df['option_names'].unique()}

# # Set up the grid layout for the subplots
# fig, axes = plt.subplots(len(unique_phenomena_subset), 1, figsize=(10, 40), sharey=True)

# # If there's only one phenomenon, we make axes an array for consistent indexing
# if len(unique_phenomena_subset) == 1:
#     axes = [axes]

# # Iterate over each phenomenon to create individual subplots
# for i, phenomenon in enumerate(unique_phenomena_subset):
#     ax = axes[i]
#     subset = grouped_subset_df[grouped_subset_df['phenomenon'] == phenomenon]
#     if not subset.empty:
#         colors = ['red' if val == subset['mean'].max() else 'gray' for val in subset['mean']]
#         sns.barplot(x='option_names', y='mean', data=subset, ax=ax, palette=palette, hue='option_names')
#         #ax.bar(subset['phenomenon'], subset['mean'], color=colors)
#         ax.set_ylabel('Mean Weighted Options')
#         ax.set_title(f"Phenomenon: {phenomenon}")
#         ax.set_xlabel('')
#         ax.set_ylabel('')

#         # Use FixedLocator to set ticks
#         ax.xaxis.set_major_locator(FixedLocator(range(len(subset['option_names'].unique()))))
#         ax.set_xticklabels(subset['option_names'].unique(), rotation=45, ha="right")
#         #ax.grid(True, axis='y')

#         # Remove all lines on background
#         ax.grid(False)

#         # Increase tick label size
#         ax.tick_params(axis="x", rotation=90, labelsize=14)
#         ax.tick_params(axis="y", labelsize=14)
        
#         # Increase axis label size
#         ax.set_xlabel(ax.get_xlabel(), fontsize=14)
#         ax.set_ylabel(ax.get_ylabel(), fontsize=14)

# # Adjust layout
# fig.tight_layout()
# plt.show()


In [54]:
# # Creating a proper name for the plot
# plot_name = "rating_w.score_gpt_possible.pdf"
# plot_path = os.path.join("plots", plot_name)

# # Ensure the plots directory exists
# os.makedirs(os.path.dirname(plot_path), exist_ok=True)

# # Save the plot
# fig.savefig(plot_path)

# plot_path
